In [ ]:
# Parámetros e imports

from pathlib import Path
import numpy as np
import pandas as pd
import librosa

# Carpeta con los .wav (una subcarpeta por especie)
IN_DIR = "../data/Tinamidae"

# Archivo de salida
OUT_CSV = "../traits/output_csv/A.csv"

# Parámetros de audio y espectrograma Mel
SR_TARGET  = 44100      # Hz
WIN_LENGTH = 1024       # muestras (ventana Hann)
HOP_LENGTH = WIN_LENGTH // 2
N_FFT      = 2048
N_MELS     = 128
FMIN, FMAX = 0.0, None
POWER      = 2.0        # potencia (magnitud^2)

# Estabilidad numérica
EPS = 1e-12

In [ ]:
# Funciones auxiliares de IO y preprocesamiento

def list_wavs(base_dir: str):
    base = Path(base_dir)
    return sorted([p for p in base.rglob("*.wav") if p.is_file()])

def species_from_path(p: Path, base_dir: str):
    """
    Asume estructura base_dir/especie/archivo.wav
    Devuelve el nombre de la carpeta inmediatamente bajo base_dir.
    """
    base = Path(base_dir).resolve()
    p = p.resolve()
    rel = p.relative_to(base)
    return rel.parts[0] if len(rel.parts) > 1 else "unknown"

def load_mono(path: Path, sr_target: int):
    """
    Carga el audio como mono y lo re-muestrea a sr_target.
    Devuelve: y (np.ndarray), sr_target
    """
    y, sr = librosa.load(path, sr=None, mono=True)
    if sr != sr_target:
        y = librosa.resample(y, orig_sr=sr, target_sr=sr_target)
        sr = sr_target
    return y.astype(np.float64, copy=False), sr

def zscore_signal(y: np.ndarray, eps: float = EPS):
    """
    Estandariza la señal: (y - media) / sigma.
    El coeficiente de variación A es invariante al re-escalado,
    pero zscore deja todo en un rango comparable.
    """
    if y.size == 0:
        return y
    mu = float(np.mean(y))
    sigma = float(np.std(y, ddof=0))
    return (y - mu) / (sigma + eps)


In [ ]:
# Funciones para Mel, RMS_t y A (coeficiente de variación de amplitud)

def melspec_power(y: np.ndarray, sr: int):
    """
    Espectrograma Mel de potencia lineal:
    S[f, t]^2 = potencia en banda Mel f y frame t.
    """
    S = librosa.feature.melspectrogram(
        y=y, sr=sr,
        n_fft=N_FFT, hop_length=HOP_LENGTH, win_length=WIN_LENGTH,
        window="hann", center=True, power=POWER,
        n_mels=N_MELS, fmin=FMIN, fmax=FMAX
    )
    return S  # matriz de potencia (F x T)

def rms_series_from_mel_power(P: np.ndarray):
    """
    Serie temporal RMS_t a partir de la potencia Mel:
    RMS_t = sqrt( (1/F) * sum_f P[f, t] ).
    """
    if P.size == 0:
        return np.zeros(0, dtype=float)
    return np.sqrt(np.mean(P, axis=0))

def compute_A_from_rms(rms_t: np.ndarray):
    """
    Coeficiente de variación de amplitud A:
    A = sigma(RMS_t) / mu(RMS_t)
    usando desviación estándar muestral (ddof=1).
    """
    if rms_t.size == 0:
        return np.nan
    mean = float(np.mean(rms_t))
    if rms_t.size > 1:
        std = float(np.std(rms_t, ddof=1))
    else:
        std = 0.0
    return std / (mean + EPS)


In [ ]:
# Cálculo de A crudo (A) por archivo

files = list_wavs(IN_DIR)
print(f"Archivos .wav encontrados: {len(files)}")

rows = []

for fp in files:
    especie = species_from_path(fp, IN_DIR)
    try:
        y, sr = load_mono(fp, SR_TARGET)
        y_hat = zscore_signal(y)
        Pmel  = melspec_power(y_hat, sr)
        rms_t = rms_series_from_mel_power(Pmel)

        A_raw = compute_A_from_rms(rms_t)
        T_frames = int(rms_t.size)

        rows.append({
            "species": especie,
            "relpath": str(fp.resolve()),
            "sr": sr,
            "T_frames": T_frames,
            "A_raw": A_raw,
        })
    except Exception as e:
        print(f"[WARN] Error en {fp}: {e}")

df_archivos = pd.DataFrame(rows)
print("Filas válidas:", df_archivos.shape[0])
df_archivos.head()


In [ ]:
# Corrección logarítmica por duración: Â = A - (a + b * log T)
# Aquí estimamos a y b por mínimos cuadrados ordinarios (MCO)
# y mostramos en pantalla los valores obtenidos.

df_dur = df_archivos.dropna(subset=["A_raw", "T_frames"]).copy()
df_dur = df_dur[df_dur["T_frames"] > 0].copy()

df_dur["log_T"] = np.log(df_dur["T_frames"].astype(float))

# Ajuste lineal: A_raw ~ a + b * log(T_frames)
x = df_dur["log_T"].values
y = df_dur["A_raw"].values

b, a = np.polyfit(x, y, 1)  # y ≈ a + b*x

print(f"Coeficientes de corrección (MCO) usando N = {len(x)} audios:")
print(f"  a (intercepto) = {a:.6f}")
print(f"  b (pendiente)  = {b:.6f}")

# Aplicamos la corrección: Â = A - (a + b * log T)
df_dur["A_hat"] = df_dur["A_raw"] - (a + b * df_dur["log_T"])

print("\nEjemplo de primeras filas con A y A_hat:")
display(df_dur[["species", "T_frames", "A_raw", "A_hat"]].head())

In [ ]:
# Resumen por especie y exporte de A.csv

df_especie = (
    df_dur
    .dropna(subset=["A_raw", "A_hat"])
    .groupby("species")
    .agg(
        n_muestras   = ("relpath", "count"),
        T_frames_mean = ("T_frames", "mean"),
        A_raw_mean   = ("A_raw", "mean"),
        A_raw_median = ("A_raw", "median"),
        A_hat_mean   = ("A_hat", "mean"),
        A_hat_median = ("A_hat", "median"),
    )
    .reset_index()
    .sort_values("n_muestras", ascending=False)
)

print("Especies con datos:", df_especie.shape[0])
df_especie.head(10)

df_especie.to_csv(OUT_CSV, index=False)
print(f"[OK] Archivo guardado en: {OUT_CSV}")
